In [1]:
import numpy as np
%matplotlib inline
from pylab import *

import scipy.integrate
import scipy




from collections import namedtuple

import numpy as np
%matplotlib inline
from pylab import *

import astropy
from astropy import units as ur
from astropy import constants as cc
import astropy.modeling.blackbody



## Introduction


Here I tabulate the Étendue of various systems 

$\Omega \approx \pi \sin^2 \theta$ and this is equivalent to $\Omega \approx \frac{\pi}{(2\cdot f/\#)^2}$

$G_{telescope} = \Omega_{telescope} \cdot A_{telescope} \cdot \epsilon_{telescope} $

In [2]:
def theta_to_omega(theta_rad):
    
    return np.pi * np.sin(theta_rad)**2 * ur.steradian

def fnum_to_omega(fnum):
    return np.pi /(2*fnum)**2 * ur.steradian

In [72]:
# Useful constants
nm = ur.nm
µm = ur.micrometer
sr = ur.steradian
hc = cc.h * cc.c



# For the system dictionary
leff = "leff" # Effective wavelength
efficiency = "efficiency" # Total efficiency of spectrograph slit - to - detector
bandwidth = "bandwidth" # Slit width of spectrrograph in linear units
pixel_pitch = "pixel_pitch" # Distance of pixels in micron 
focal_ratio = "focal_ratio" # f/# of the spectrograph camera
agl = "agl" # Airplane altitude
ground_speed = "ground_speed" # ground speed
objective_ø = "objective_ø" # Diameter of objective

In [40]:
Reflectivity_Canopy = 0.5 # This is the light reflected back up

Leff = 700 * nm
EPP = (hc/Leff).to(ur.erg) # Energy per photon

In [41]:
EPP

<Quantity 2.83777975e-12 erg>

In [54]:
def print_system(system):
    
    print(system)

In [95]:
def system_to_grasp(system, loud=False):
    
    Omega = fnum_to_omega(system[focal_ratio])
    grasp = (system[pixel_pitch]**2 * Omega).to(µm**2 * sr)
    
    if loud: print("System grasp is %s" % grasp)
        
    return grasp

def system_to_fl(system):
    """ Focal length of system """
    
    return system[objective_ø] * system[focal_ratio]

def system_to_gsd(system):
    """ Ground sampling distance of system
    GSD is the pixel-to-pixel distance in meter
    nb GSD is not the same as ground resolution which will be several GSDs"""
    
    fl = system_to_fl(system)
    theta = system[pixel_pitch]/fl
    return (system[agl] * theta).to(ur.meter)

def system_to_grd(system):
    """ Ground resolution distance """
    
    return system_to_gsd(system)*3

def system_to_et(system):
    """ Time it takes airplane to move one slit distance
    (exposure time)"""
    
    dist = system_to_grd(system)
    speed = system[ground_speed]
    
    return (dist/speed).to(ur.second)

In [96]:
system_to_et(system)

<Quantity 0.1875 s>

In [123]:


system ={leff: 700*nm, efficiency: 0.5, bandwidth: 0.05*nm, pixel_pitch: 15 * µm,
        focal_ratio: 2.0, agl: 1000*ur.meter, ground_speed: 60*ur.meter/ur.second,
        objective_ø: 2*ur.mm}


print_system(system)
grasp = system_to_grasp(system)
print(grasp)
f_lambda = 1300*ur.Watt/ur.meter**2/ur.micron/(2*np.pi*ur.sr) # http://www.powerfromthesun.net/Book/chapter02/chapter02.html

photon_per_second = (f_lambda / EPP * system[bandwidth] * grasp).cgs

print()
SW = (system_to_gsd(system) * 500).to(ur.meter)
print("Swath width at 500 pix %s" % (SW))

print()

print("Number photons received per et %4.0f" % (photon_per_second * system_to_et(system)))
print("Corresponding SNR limit %4.")

{'leff': <Quantity 700. nm>, 'efficiency': 0.5, 'bandwidth': <Quantity 0.05 nm>, 'pixel_pitch': <Quantity 15. um>, 'focal_ratio': 2.0, 'agl': <Quantity 1000. m>, 'ground_speed': <Quantity 60. m / s>, 'objective_ø': <Quantity 2. mm>}
44.178646691106465 sr um2

Swath width at 500 pix 1875.0 m

Number photons received per et 301973
